In [ ]:
##### review file code

# select max( mthid )from dwh_rel.td_account_details_hist
# order by mthid

# -- select top 10 * from mydb.dsg_portfolio.rel_td_hist_maytodec

# unload ($$ 
# with 
# acqui_base as (
# select distinct a.crn,a.accnt_num, accnt_opn_date::date as accnt_opn_date, 
# accnt_maturity_date::date as accnt_maturity_date ,accnt_cls_date::date as accnt_cls_date ,rnwal_date,
# initial_depst_amt, accnt_bal_in_bc,  prodct_code,mthid,tenor_in_days
# from
# --mydb.dsg_portfolio.rel_td_hist_maytodec  a
# --dsg_portfolio.rel_td_account_details a
# prod_db.dwh_rel.td_account_details_hist a
# where  
#  td_cust_clssfcaton_code in ('KJIFI','KNOW','KSEL','K_NOB')
# and mthid in (202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,
# 202311,202312)

# ),
# fkyc_base as (
# select distinct a.crn,b.accnt_num, fkyc_date 
# from 
# (select crn, max(fkyc_date) as fkyc_date from 
# dsg_portfolio.fkyc_base_ntb
# where fkyc_date is not null
# group by crn
# )
# a
# join acqui_base b 
# on a.crn::bigint=b.crn::bigint
# where (b.accnt_opn_date-fkyc_date)<=365
# ),
# scc_base as (
# Select distinct APPL_USER_2,date_opened_g from dwh_STG.AMBS_STG
# where logo = 420
# ---AND cast (date_opened_g as date) >= '2024-01-01'
# and APPL_USER_2 is not NULL
# and APPL_USER_2 !=''

# ),
# app_td as (
# select distinct td_id as accnt_num  
# from
# app_811_onb_prod_fixed_deposit.tbl_term_deposit a 
#       where source='811APP'
#     and account_id!=''

# )
# select a.*,fkyc_date,APPL_USER_2,date_opened_g,d.accnt_num as app_accnt_num
# from acqui_base a 
# left join fkyc_base b 
# on a.accnt_num=b.accnt_num 
# left join scc_base c 
# on a.accnt_num::float=c.APPL_USER_2::bigint
# left join app_td d 
# on a.accnt_num=d.accnt_num
# $$
# )
# to 's3://kotak-811-cdp-peak-write/811-cdp-peak-main/datascience/td/reports/jan23_dec23/'
# IAM_ROLE
# 'arn:aws:iam::718378052708:role/service-role/AmazonRedshift-CommandsAccessRole-20240513T161101'
# FORMAT PARQUET
# parallel off;



In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import datetime
from dateutil.relativedelta import relativedelta


In [13]:
# df_parent=pd.read_parquet("/Users/kmbl227932/Downloads/jantoaprandjan25acqui.parquet")
# # df_parent['a]
# df_parent1=pd.read_parquet("/Users/kmbl227932/Downloads/may24.parquet")
# df_parent_2=pd.read_parquet("/Users/kmbl227932/Downloads/maytodecacqui.parquet")
# df_parent=pd.concat([df_parent,df_parent1,df_parent_2],axis=0)


# df_parent=pd.read_parquet("/Users/kmbl227932/Downloads/full_jan24_feb25.parquet")
# df_parent=pd.read_parquet("/Users/kmbl227932/Downloads/jan23_dec23.parquet")
df_parent=pd.read_parquet("/Users/kmbl227932/Downloads/feb_mar_25.parquet")



In [14]:
df_parent.columns

Index(['crn', 'accnt_num', 'accnt_opn_date', 'accnt_maturity_date',
       'accnt_cls_date', 'rnwal_date', 'initial_depst_amt', 'accnt_bal_in_bc',
       'prodct_code', 'mthid', 'tenor_in_days', 'fkyc_date', 'appl_user_2',
       'date_opened_g', 'app_accnt_num'],
      dtype='object')

In [15]:
df_parent.mthid.value_counts()

mthid
202503    11850307
202502    11366441
Name: count, dtype: int64

In [16]:
df_parent.columns

Index(['crn', 'accnt_num', 'accnt_opn_date', 'accnt_maturity_date',
       'accnt_cls_date', 'rnwal_date', 'initial_depst_amt', 'accnt_bal_in_bc',
       'prodct_code', 'mthid', 'tenor_in_days', 'fkyc_date', 'appl_user_2',
       'date_opened_g', 'app_accnt_num'],
      dtype='object')

In [18]:
df_parent['initial_depst_amt'].fillna(0,inplace=True)
df_parent['accnt_bal_in_bc'].fillna(0,inplace=True)

df_parent['initial_depst_amt']=df_parent['initial_depst_amt'].replace('',0)
df_parent['initial_depst_amt']=df_parent['initial_depst_amt'].astype(float).astype(int)

df_parent['accnt_bal_in_bc']=df_parent['accnt_bal_in_bc'].replace('',0)

df_parent['accnt_bal_in_bc']=df_parent['accnt_bal_in_bc'].astype(float).astype(int)




/var/folders/ht/mft9wz317xd4cttlzy2ntw403_jv97/T/ipykernel_30483/2626429918.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_parent['initial_depst_amt'].fillna(0,inplace=True)


In [19]:
df_parent.columns=df_parent.columns.str.upper()


In [20]:
df_parent.MTHID.unique()

array(['202502', '202503'], dtype=object)

In [21]:
df_parent['MTHID']=df_parent['MTHID'].astype(int)

In [22]:
df_parent['ACCNT_OPN_DATE']=pd.to_datetime(df_parent['ACCNT_OPN_DATE'])
df_parent['ACCNT_CLS_DATE']=pd.to_datetime(df_parent['ACCNT_CLS_DATE'])
df_parent['ACCNT_MATURITY_DATE']=pd.to_datetime(df_parent['ACCNT_MATURITY_DATE'])
df_parent['FKYC_DATE']=pd.to_datetime(df_parent['FKYC_DATE'])

In [53]:
#inflow/ outflow value

df_parent.columns=df_parent.columns.str.upper()

df_value=pd.DataFrame(columns=['Year','Month','Acqui','AR_planned','On_maturity','Pre Cls_same_mnth','Pre Cls_prev_mnth','Pre Cls_older'])


for i in df_parent.MTHID.unique():
    print(i)
    mon=i%100
    yr=int(i/100)
    print(mon)
    print(yr)

    #################filtering for 811 app FDs###########################
    # mthid=yr*100+mon
    df=df_parent[(df_parent['MTHID']==i)]
    # df_parent.columns=df_parent.columns.str.upper()
    
    #### for FD
    
    
    ##Only for ETB- for others- comment SCC line and fkyc not needed
    df=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]
    # df=df[(df['PRODCT_CODE'].str.contains('FF'))]
    
    #remove SCC if needed
    # df=df[~df['ACCNT_NUM'].isin(scc.APPL_USER_2.unique())]


    
    
    df=df[df['INITIAL_DEPST_AMT']<20000000]
    df['ACCNT_OPN_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_OPN_DATE']),axis=1)
    df['ACCNT_CLS_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_CLS_DATE']),axis=1)
    df['RNWAL_DATE']=pd.to_datetime(df['RNWAL_DATE'])
    df['ACCNT_MATURITY_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_MATURITY_DATE']),axis=1)
    df=df.drop_duplicates('ACCNT_NUM')
    
    
    # df_count=pd.DataFrame(columns=['Year','Month','Acqui','AR_planned','AR_unplanned','On_maturity',' Pre Cls_same_mnth','Pre Cls_prev_mnth','Pre Cls_older'])
    
    # df=pd.merge(df,fkyc,left_on='CRN', right_on='crn',how='left')
    # df=df[(df['fkyc_date'].isna())|(df['fkyc_date']<fkyc_limit)]
    
    
    
    
    
    #Acquisition	opn date in that month init amt sum 	later subtract renewal unplanned
    acqui=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon)]
    # acqui=acqui[(acqui['fkyc_date'].isna())|(acqui['fkyc_date']<fkyc_limit)]
    
    acq_crn=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon)]['CRN'].unique()
    
    #Renewal - Planned 	renewal date in that month and rnwl flag=y and  renewal flag= U, renewal date >=
    #for new accnts, renewal date is similar to accnt_opn_date- so removing those acccnts as well
    renew_pl=df[(df['RNWAL_DATE'].dt.year==yr)
                &(df['RNWAL_DATE'].dt.month==mon)
                &(df['ACCNT_OPN_DATE']!=df['RNWAL_DATE'])
    
                &(df['ACCNT_MATURITY_DATE']>df['RNWAL_DATE'])
                &((df['ACCNT_CLS_DATE'].isna()))
    ]['INITIAL_DEPST_AMT'].sum()
    
    #Renewal - Unplanned 	maturity date in same month and crn in acqui of the same month	
    # crn_renew_unpl=df[(df['ACCNT_MATURITY_DATE'].dt.year==yr)
    #                   &(df['ACCNT_MATURITY_DATE'].dt.month==mon)
    #                   &(df['AUTO_RNWAL_FLAG']=='N')&(df['CRN'].isin(acq_crn))&(~df['ACCNT_CLS_DATE'].isna())]['CRN'].unique()
    # crn_renew_unpl=df[(df['ACCNT_OPN_DATE'].dt.year==yr)
    #                   &(df['ACCNT_OPN_DATE'].dt.month==mon)&(df['CRN'].isin(crn_renew_unpl))]['CRN'].unique()
    # renew_unpl=df[(df['ACCNT_OPN_DATE'].dt.year==yr)
    #                   &(df['ACCNT_OPN_DATE'].dt.month==mon)&df['CRN'].isin(crn_renew_unpl)]['INITIAL_DEPST_AMT'].sum()
    print(acqui.shape)
    #     #subtract unplanned renew from  acqui
    #     acqui=acqui[~(acqui['CRN'].isin(crn_renew_unpl))]
    acqui=acqui['INITIAL_DEPST_AMT'].sum()
    
    #Closure - Same month 	opn date and cls date issame month	
    cl_sm=df[((df['ACCNT_OPN_DATE'].dt.year==df['ACCNT_CLS_DATE'].dt.year)
              &(df['ACCNT_CLS_DATE'].dt.year==yr))
              &(df['ACCNT_MATURITY_DATE'].dt.date>df['ACCNT_CLS_DATE'].dt.date)
             &((df['ACCNT_OPN_DATE'].dt.month==df['ACCNT_CLS_DATE'].dt.month)&(df['ACCNT_CLS_DATE'].dt.month==mon))]['INITIAL_DEPST_AMT'].sum()
    
    #Closure - Previous Month  month 	opn month prev cls month curr	(for now works for mar to dec ->> month-1 does not work for dec)
    cl_pm=df[((df['ACCNT_CLS_DATE'].dt.year==yr))
               &(df['ACCNT_CLS_DATE'].dt.month==mon)
              &(df['ACCNT_MATURITY_DATE']>df['ACCNT_CLS_DATE'])
    #          Cannot compare Timestamp with datetime.date
              &( df['ACCNT_OPN_DATE']<(datetime.datetime(yr,mon,1,0,0)- pd.DateOffset(months=0)))
              &( df['ACCNT_OPN_DATE']>=(datetime.datetime(yr,mon,1,0,0)- pd.DateOffset(months=1)))
    #          &(df['ACCNT_OPN_DATE'].dt.month==(mon-1))
           ]['INITIAL_DEPST_AMT'].sum()
    
    
    
    
    #Closure from old base 	opn month any month cls month curr	
    cl_old=df[(df['ACCNT_CLS_DATE'].dt.year==yr) &(df['ACCNT_CLS_DATE'].dt.month==mon)
               &(df['ACCNT_MATURITY_DATE']>df['ACCNT_CLS_DATE'])
              # add year as well here- as only month would remove the last year accnts as well
              
             &( (df['ACCNT_OPN_DATE']<(datetime.datetime(yr,mon,1,0,0)- pd.DateOffset(months=1))))
    #           &(~(df['ACCNT_OPN_DATE'].dt.month.isin([mon,mon-1]) & df.['ACCNT_OPN_DATE'].dt.year==yr))
              ]['INITIAL_DEPST_AMT'].sum()
    
    
    #Maturity
    mat=df[((df['ACCNT_MATURITY_DATE'].dt.year==df['ACCNT_CLS_DATE'].dt.year)
            &(df['ACCNT_CLS_DATE'].dt.year==yr))
          & (df['ACCNT_MATURITY_DATE']==df['ACCNT_CLS_DATE'])
           &((df['ACCNT_MATURITY_DATE'].dt.month==df['ACCNT_CLS_DATE'].dt.month)
             &(df['ACCNT_CLS_DATE'].dt.month==mon))]['INITIAL_DEPST_AMT'].sum()
    
    df_value.loc[len(df_value.index)] = [yr,mon,acqui,renew_pl,mat,cl_sm,cl_pm,cl_old]
    


202502
2
2025
(65797, 15)
202503
3
2025
(75996, 15)


In [54]:
df_value

,Year,Month,Acqui,AR_planned,On_maturity,Pre Cls_same_mnth,Pre Cls_prev_mnth,Pre Cls_older
0,2025,2,4285650199,1309131880,736197370,637653044,440874936,1192875142
1,2025,3,4703329855,1619742073,911825763,697162308,482929500,1380700284


In [27]:
df_value=df_value.sort_values(['Year','Month']).reset_index(drop=True)

In [55]:
(df_value[['Acqui','AR_planned', 
       'On_maturity', 'Pre Cls_same_mnth', 'Pre Cls_prev_mnth', 'Pre Cls_older']]/10000000).T

,0,1
Acqui,428.565,470.333
AR_planned,130.913,161.974
On_maturity,73.620,91.183
Pre Cls_same_mnth,63.765,69.716
Pre Cls_prev_mnth,44.087,48.293
Pre Cls_older,119.288,138.070


In [62]:
# df=df_parent[(df_parent['MTHID']==202501)&(~df_parent['APP_ACCNT_NUM'].isna())]
df=df_parent[(df_parent['MTHID']==202501)]

df.columns=df.columns.str.upper()
df=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]  #FD
# df=df[(~df['PRODCT_CODE'].str.contains('FF'))&(~(df['PRODCT_CODE'].str.contains('RD')))]
# df=df[df['PRODCT_CODE'].str.contains('RD')]
df=df[df['INITIAL_DEPST_AMT']<20000000]

df['ACCNT_OPN_DATE']=pd.to_datetime(df['ACCNT_OPN_DATE'])
df['ACCNT_CLS_DATE']=pd.to_datetime(df['ACCNT_CLS_DATE'])
df['RNWAL_DATE']=pd.to_datetime(df['RNWAL_DATE'])
df['ACCNT_MATURITY_DATE']=pd.to_datetime(df['ACCNT_MATURITY_DATE'])
df=df.drop_duplicates('ACCNT_NUM')



In [63]:
df

,CRN,ACCNT_NUM,ACCNT_OPN_DATE,ACCNT_MATURITY_DATE,ACCNT_CLS_DATE,RNWAL_DATE,INITIAL_DEPST_AMT,ACCNT_BAL_IN_BC,PRODCT_CODE,MTHID,TENOR_IN_DAYS,FKYC_DATE,APPL_USER_2,DATE_OPENED_G,APP_ACCNT_NUM
3,796169734,9649080626,2024-03-10,2026-02-09,NaT,NaT,90000,95097,TRTCUM,202501,701,2023-11-03,9649080626,2024-03-10,None
178,637577837,3747648245,2022-12-22,2023-06-20,2023-06-20,NaT,10000,0,TRT180,202501,180,2022-08-17,None,NaT,None
226,603074043,0347731797,2023-01-28,2023-02-27,2023-02-01,NaT,20000,0,TRT180,202501,30,2022-06-04,None,NaT,None
227,389848570,8945157759,2021-06-05,2022-07-05,2021-06-30,NaT,10000,0,TRTMPO,202501,395,None,None,NaT,None
305,667769055,6749127114,2024-03-27,2025-03-22,2025-01-04,2024-09-23,310328,0,TRT180,202501,180,None,None,NaT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120839386,554536577,5447008800,2022-07-29,2023-07-29,2022-07-29,NaT,10000,0,TRTQPO,202501,365,2022-01-14,None,NaT,None
120839387,600711083,0048041171,2023-03-11,2024-03-05,2023-09-07,2023-09-07,51603,0,TRT180,202501,180,2022-11-07,None,NaT,None
120839394,120291889,2013127293,2018-01-29,2018-07-26,2018-07-26,NaT,10000,0,TRT180,202501,180,2017-10-07,None,NaT,None
120839573,707872472,0750738635,2024-10-27,2024-11-16,2024-11-06,2024-11-06,24318,0,TRT180,202501,10,None,None,NaT,None


In [64]:
df_parent.TENOR_IN_DAYS.value_counts(normalize=True).head(10)

TENOR_IN_DAYS
180   0.494
365   0.064
366   0.053
390   0.039
184   0.039
182   0.030
181   0.030
183   0.028
391   0.024
367   0.012
Name: proportion, dtype: float64

In [65]:
df_parent.TENOR_IN_DAYS.value_counts(normalize=True).head(10)

TENOR_IN_DAYS
180   0.494
365   0.064
366   0.053
390   0.039
184   0.039
182   0.030
181   0.030
183   0.028
391   0.024
367   0.012
Name: proportion, dtype: float64

In [66]:
df.TENOR_IN_DAYS.value_counts(normalize=True).head(10)

TENOR_IN_DAYS
390   0.174
180   0.121
391   0.069
365   0.069
366   0.067
367   0.028
395   0.028
700   0.026
182   0.023
7     0.022
Name: proportion, dtype: float64

In [ ]:
# when u want to find closures, for all months, take only the latest mnthid data for all months

In [91]:
yr=2024
mon=1
df_sub_cnt=pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14])

#################### filtered for tenures #################
df_ten=df[df['TENOR_IN_DAYS']==365]

df_sub=df.copy()


for mon in range(1,13):
    # get current month here and it goes back to prev months
    curr_month=1
    curr_year=2025
    print(mon)
    # mat > cls if preclosures to be checked, mat=cls if maturity closures to be checked
    # df_sub=df_ten[df_ten['ACCNT_MATURITY_DATE']>df_ten['ACCNT_CLS_DATE']]

    df_sub=df_sub[(df_sub['ACCNT_OPN_DATE'].dt.month==mon)&(df_sub['ACCNT_OPN_DATE'].dt.year==yr)]

 

    mnth1=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).year)].shape[0]

    mnth2=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=1)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=1)).year)].shape[0]

    mnth3=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=2)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=2)).year)].shape[0]



    mnth4=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=3)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=3)).year)].shape[0]


    mnth5=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=4)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=4)).year)].shape[0]

    mnth6=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=5)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=5)).year)].shape[0]

    mnth7=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=6)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=6)).year)].shape[0]

    mnth8=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=7)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=7)).year)].shape[0]

    mnth9=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=8)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=8)).year)].shape[0]

    mnth10=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=9)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=9)).year)].shape[0]

    mnth11=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=10)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=10)).year)].shape[0]

    mnth12=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=11)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=11)).year)].shape[0]

    mnth13=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=12)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=12)).year)].shape[0]
    
    df_sub_cnt.loc[len(df_sub_cnt.index)] =[mnth1,mnth2,mnth3,mnth4,mnth5,mnth6,mnth7,mnth8,mnth9,mnth10,mnth11,mnth12,mnth13,df_sub.shape[0]]

df_sub_cnt
# print([mnth1,mnth2,mnth3,mnth4,mnth5,mnth6,mnth7,mnth8,df_sub.shape[0]])

1
2
3
4
5
6
7
8
9
10
11
12


,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,64,27,17,10,8,11,7,10,7,3,3,8,137,312
1,90,99,37,30,19,14,4,7,6,3,5,7,0,321
2,1719,626,238,160,140,108,71,79,51,60,40,0,0,3292
3,1483,494,198,121,102,85,64,60,48,38,0,0,0,2693
4,809,428,158,86,73,45,49,30,33,0,0,0,0,1711
5,764,452,159,76,55,53,38,25,0,0,0,0,0,1622
6,920,474,164,93,113,54,50,0,0,0,0,0,0,1868
7,940,453,171,116,56,42,0,0,0,0,0,0,0,1778
8,702,373,125,95,60,0,0,0,0,0,0,0,0,1355
9,604,295,133,68,1,0,0,0,0,0,0,0,0,1101


In [92]:
yr=2025
mon=1
df_sub_cnt=pd.DataFrame(columns=['count'])
# for mon in range(1,13):
df_sub=df_ten[(df_ten['ACCNT_OPN_DATE'].dt.month==mon)&(df_ten['ACCNT_OPN_DATE'].dt.year==yr)]

mnth1=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).month)
         &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).year)].shape[0]


# cnt=df[(df['ACCNT_OPN_DATE'].dt.month==mon)&(df['ACCNT_OPN_DATE'].dt.year==yr)]['ACCNT_NUM'].count()
df_sub_cnt.loc[len(df_sub_cnt.index)] =[cnt]


mnth1    

502

In [93]:
df_ten

,CRN,ACCNT_NUM,ACCNT_OPN_DATE,ACCNT_MATURITY_DATE,ACCNT_CLS_DATE,RNWAL_DATE,INITIAL_DEPST_AMT,ACCNT_BAL_IN_BC,PRODCT_CODE,MTHID,TENOR_IN_DAYS,FKYC_DATE,APPL_USER_2,DATE_OPENED_G,APP_ACCNT_NUM
389,460275296,6048118509,2023-01-08,2024-01-08,2023-01-09,NaT,5750,0,TRTMPO,202501,365,None,None,NaT,None
1163,690234010,9047843748,2023-02-09,2024-02-09,2023-06-28,NaT,5750,0,TRTMPO,202501,365,2023-02-01,9047843748,2023-02-09,None
2217,292762676,9249484732,2024-06-13,2025-06-13,2024-06-29,NaT,5000,0,TRTCUM,202501,365,None,None,NaT,None
2288,120211650,2046240921,2022-04-17,2023-04-17,2023-04-05,NaT,15000,0,TRTCUM,202501,365,None,2046240921,2022-04-21,None
2885,480670024,8047035337,2022-05-08,2023-05-08,2022-05-09,NaT,11250,0,TRTCUM,202501,365,2021-06-26,None,NaT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120834084,631793127,3146647436,2022-08-30,2023-08-30,2023-08-30,NaT,100000,0,TRTMPO,202501,365,2022-08-26,None,NaT,None
120835655,731575562,3149015782,2024-10-30,2025-10-30,2024-11-07,NaT,5000,0,TRTCUM,202501,365,None,None,NaT,None
120836354,469726335,6950329123,2024-08-08,2025-08-08,2024-08-11,NaT,15000,0,TRTCUM,202501,365,None,None,NaT,None
120836630,547999430,4746796326,2022-01-25,2023-01-25,2022-09-21,NaT,10000,0,TRTQPO,202501,365,2021-12-24,None,NaT,None


In [100]:
yr=2024
mon=1
df_sub_cnt=pd.DataFrame(columns=['count'])
df_ten['ACCNT_OPN_DATE']=pd.to_datetime(df_ten['ACCNT_OPN_DATE'])
df_ten['ACCNT_CLS_DATE']=pd.to_datetime(df_ten['ACCNT_CLS_DATE'])
df_ten['RNWAL_DATE']=pd.to_datetime(df_ten['RNWAL_DATE'])
df_ten['ACCNT_MATURITY_DATE']=pd.to_datetime(df_ten['ACCNT_MATURITY_DATE'])


for mon in range(1,13):
    cnt=df_ten[(df_ten['ACCNT_OPN_DATE'].dt.month==mon)&(df_ten['ACCNT_OPN_DATE'].dt.year==yr) ]['ACCNT_NUM'].count()
    df_sub_cnt.loc[len(df_sub_cnt.index)] =[cnt]
    
df_sub_cnt

,count
0,312
1,321
2,3292
3,2693
4,1711
5,1622
6,1868
7,1778
8,1355
9,1101


In [101]:
df_ten[(df_ten['ACCNT_OPN_DATE'].dt.month==1)&(df_ten['ACCNT_OPN_DATE'].dt.year==2025)]['ACCNT_NUM'].count()


503

In [96]:
yr=2024
mon=1
df_sub_value=pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14])



#################### filtered for tenures #################
df_ten=df[df['TENOR_IN_DAYS']==365]


# mat > cls if preclosures to be checked, mat=cls if maturity closures to be checked
df_ten=df_ten[df_ten['ACCNT_MATURITY_DATE']>df_ten['ACCNT_CLS_DATE']]


for mon in range(1,13):
    curr_month=1
    curr_year=2025

    # df_sub=df[(df['ACCNT_OPN_DATE'].dt.month==mon)&(df['ACCNT_OPN_DATE'].dt.year==yr)]
    df_sub=df_ten[(df_ten['ACCNT_OPN_DATE'].dt.month==mon)&(df_ten['ACCNT_OPN_DATE'].dt.year==yr)]

    mnth1=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth2=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=1)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=1)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth3=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=2)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=2)).year)]['INITIAL_DEPST_AMT'].sum()



    mnth4=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=3)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=3)).year)]['INITIAL_DEPST_AMT'].sum()


    mnth5=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=4)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=4)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth6=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=5)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=5)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth7=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=6)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=6)).year)]['INITIAL_DEPST_AMT'].sum()
    mnth8=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=7)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=7)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth9=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=8)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=8)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth10=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=9)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=9)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth11=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=10)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=10)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth12=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=11)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=11)).year)]['INITIAL_DEPST_AMT'].sum()

    mnth13=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=12)).month)
             &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=12)).year)]['INITIAL_DEPST_AMT'].sum()



#     df_sub_cnt.loc[len(df_sub_cnt.index)] =[mnth1,mnth2,mnth3,mnth4,mnth5,mnth6,mnth7,mnth8,mnth9,mnth10,mnth11,mnth12,mnth13,df_sub['INITIAL_DEPST_AMT'].sum()]



    df_sub_value.loc[len(df_sub_value.index)] =[mnth1,mnth2,mnth3,mnth4,mnth5,mnth6,mnth7,mnth8,mnth9,mnth10,mnth11,mnth12,mnth13,df_sub['INITIAL_DEPST_AMT'].sum()]


# print([mnth1,mnth2,mnth3,mnth4,mnth5,mnth6,mnth7,mnth8,df_sub.shape[0]])
df_sub_value/10000000

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.292,0.123,0.074,0.043,0.013,0.069,0.027,0.025,0.164,0.062,0.002,0.034,0.607,1.534
1,0.157,0.327,0.082,0.097,0.030,0.224,0.015,0.041,0.024,0.009,0.023,0.039,0.000,1.067
2,3.577,1.830,1.128,0.645,0.436,0.326,0.217,0.317,0.143,0.244,0.123,0.000,0.000,8.985
3,3.301,1.614,0.594,0.394,0.284,0.193,0.278,0.152,0.115,0.268,0.000,0.000,0.000,7.194
4,1.888,1.479,0.686,0.653,0.449,0.193,0.656,0.201,0.245,0.000,0.000,0.000,0.000,6.450
5,1.984,1.341,0.505,0.370,0.246,0.255,0.247,0.109,0.000,0.000,0.000,0.000,0.000,5.055
6,3.568,2.204,0.618,0.457,0.338,0.253,0.261,0.000,0.000,0.000,0.000,0.000,0.000,7.698
7,3.147,1.687,0.549,0.389,0.252,0.379,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6.402
8,2.580,1.211,0.307,0.492,0.399,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.989
9,2.051,0.573,0.536,0.195,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.355


In [97]:
yr=2025
mon=1
df_sub_cnt=pd.DataFrame(columns=['count'])
# for mon in range(1,13):
df_sub=df_ten[(df_ten['ACCNT_OPN_DATE'].dt.month==mon)&(df_ten['ACCNT_OPN_DATE'].dt.year==yr)]

mnth1=df_sub[(df_sub['ACCNT_CLS_DATE'].dt.month==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).month)
         &(df_sub['ACCNT_CLS_DATE'].dt.year==(datetime.datetime(yr,mon,1,0,0)+ pd.DateOffset(months=0)).year)]['INITIAL_DEPST_AMT'].sum()


# cnt=df[(df['ACCNT_OPN_DATE'].dt.month==mon)&(df['ACCNT_OPN_DATE'].dt.year==yr)]['ACCNT_NUM'].count()
df_sub_cnt.loc[len(df_sub_cnt.index)] =[cnt]


mnth1/10000000  

1.2193224

In [98]:
yr=2024
mon=1
df_sub_cnt=pd.DataFrame(columns=['count'])

df_sub_cnt=pd.DataFrame(columns=['count'])
df['ACCNT_OPN_DATE']=pd.to_datetime(df['ACCNT_OPN_DATE'])
df['ACCNT_CLS_DATE']=pd.to_datetime(df['ACCNT_CLS_DATE'])
df['RNWAL_DATE']=pd.to_datetime(df['RNWAL_DATE'])
df['ACCNT_MATURITY_DATE']=pd.to_datetime(df['ACCNT_MATURITY_DATE'])


for mon in range(1,13):
    cnt=df[(df['ACCNT_OPN_DATE'].dt.month==mon)&(df['ACCNT_OPN_DATE'].dt.year==yr) & (df['TENOR_IN_DAYS']==365)]['INITIAL_DEPST_AMT'].sum()
    df_sub_cnt.loc[len(df_sub_cnt.index)] =[cnt]
    
df_sub_cnt/10000000

,count
0,5.077
1,2.214
2,13.659
3,12.194
4,9.385
5,8.585
6,12.757
7,10.552
8,7.899
9,6.208


In [99]:
df[(df['ACCNT_OPN_DATE'].dt.month==1)&(df['ACCNT_OPN_DATE'].dt.year==2025)& (df['TENOR_IN_DAYS']==365)]['INITIAL_DEPST_AMT'].sum()


57109840

In [ ]:
df_parent['ACCNT_CLS_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_CLS_DATE']),axis=1)
df_parent['RNWAL_DATE']=pd.to_datetime(df['RNWAL_DATE'])
df_parent['ACCNT_MATURITY_DATE']=df_parent.apply(lambda x:pd.Timestamp(x['ACCNT_MATURITY_DATE']),axis=1)


In [9]:
df_parent['ACCNT_OPN_DATE']=pd.to_datetime(df_parent['ACCNT_OPN_DATE'])

In [10]:
df_parent['ACCNT_CLS_DATE']=pd.to_datetime(df_parent['ACCNT_CLS_DATE'])

In [17]:
df_parent['ACCNT_MATURITY_DATE']=pd.to_datetime(df_parent['ACCNT_MATURITY_DATE'])

In [10]:
df_parent.columns

Index(['CRN', 'ACCNT_NUM', 'ACCNT_OPN_DATE', 'ACCNT_MATURITY_DATE',
       'ACCNT_CLS_DATE', 'RNWAL_DATE', 'INITIAL_DEPST_AMT', 'ACCNT_BAL_IN_BC',
       'PRODCT_CODE', 'MTHID', 'TENOR_IN_DAYS', 'FKYC_DATE', 'APPL_USER_2',
       'DATE_OPENED_G', 'APP_ACCNT_NUM'],
      dtype='object')

In [11]:
df_parent.MTHID.unique()

array([202311, 202303, 202309, 202302, 202305, 202310, 202304, 202308,
       202307, 202306, 202312, 202301], dtype=int32)

In [31]:
df_parent

,CRN,ACCNT_NUM,ACCNT_OPN_DATE,ACCNT_MATURITY_DATE,ACCNT_CLS_DATE,RNWAL_DATE,INITIAL_DEPST_AMT,ACCNT_BAL_IN_BC,PRODCT_CODE,MTHID,TENOR_IN_DAYS,FKYC_DATE,APPL_USER_2,DATE_OPENED_G,APP_ACCNT_NUM
0,413526790,1346078913,2022-05-11,2023-05-11,2023-05-11,None,1000,0,TRDGEN,202311,365.000,None,None,NaT,None
1,202987890,9042884937,2022-02-07,2022-08-06,2022-02-08,None,20000,0,TFFDRT,202311,180.000,None,None,NaT,None
2,65477534,3450666730,2023-02-05,2023-08-04,NaT,None,115000,70000,TFFJIF,202303,180.000,None,None,NaT,None
3,363352432,6345950581,2021-07-07,2026-08-06,NaT,None,10000,11110,TRTCUM,202309,1856.000,None,None,NaT,None
4,362856190,6247985575,2022-11-16,2023-11-16,NaT,None,5750,5750,TRTMPO,202309,365.000,None,6247985575,2022-11-19,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44353166,74033460,6010736213,2016-03-29,2016-09-25,2016-03-30,None,10000,0,TFFJIF,202304,180.000,2017-11-12,None,NaT,None
44353167,487589832,8746022171,2021-09-06,2031-09-06,2021-12-11,None,100,0,TRDGEN,202306,3652.000,2021-08-21,None,NaT,None
44353168,468003888,6845977910,2021-07-09,2025-07-11,2023-08-13,2023-07-10,11046,0,TRTCUM,202311,732.000,2021-03-15,None,NaT,None
44353169,313540686,1313854076,2019-05-10,2020-02-10,2019-06-05,None,2500,0,TRDGEN,202309,276.000,2019-02-03,None,NaT,None


In [24]:
#NTB /ETB Split Acqui

df_parent['FKYC_DATE']=pd.to_datetime(df_parent['FKYC_DATE'])
df_parent.columns=df_parent.columns.str.upper()

df_value=pd.DataFrame(columns=['Year','Month','Acqui_ntb_0_1_cnt','Acqui_ntb_0_1_val','Acqui_ntb_2_12_cnt','Acqui_ntb_2_12_val','Acqui_eTB_cnt','Acqui_eTB_val','SCC cnt', 'SCC val'])


for i in df_parent.MTHID.unique():
    print(i)
    mon=i%100
    yr=int(i/100)
    print(mon)
    print(yr)

    # mthid=yr*100+mon
    df=df_parent[(df_parent['MTHID']==i) ]

    
    ##Only for ETB- for others- comment SCC line and fkyc not needed
    # df=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]
    df=df[(df['PRODCT_CODE'].str.contains('FF'))]

    #remove SCC in needed
    # df=df[~df['ACCNT_NUM'].isin(scc.APPL_USER_2.unique())]
    
    df=df[df['INITIAL_DEPST_AMT']<20000000]
    
    # df['ACCNT_OPN_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_OPN_DATE']),axis=1)
    # df['ACCNT_CLS_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_CLS_DATE']),axis=1)
    # df['RNWAL_DATE']=pd.to_datetime(df['RNWAL_DATE'])
    # df['ACCNT_MATURITY_DATE']=df.apply(lambda x:pd.Timestamp(x['ACCNT_MATURITY_DATE']),axis=1)
    df=df.drop_duplicates('ACCNT_NUM')
    
    
    # df_count=pd.DataFrame(columns=['Year','Month','Acqui','AR_planned','AR_unplanned','On_maturity',' Pre Cls_same_mnth','Pre Cls_prev_mnth','Pre Cls_older'])
    
    # df=pd.merge(df,fkyc,left_on='CRN', right_on='crn',how='left')
    # df=df[(df['fkyc_date'].isna())|(df['fkyc_date']<fkyc_limit)]
    
    
    
    
    
    #Acquisition	opn date in that month init amt sum 	later subtract renewal unplanned
    acqui=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon) &(df['APPL_USER_2'].isna())]

    print(acqui.shape)
    ntb=acqui[~acqui['FKYC_DATE'].isna()]
    
    ntb_1mnth=ntb[(ntb['ACCNT_OPN_DATE'].dt.month==ntb['FKYC_DATE'].dt.month)&((ntb['ACCNT_OPN_DATE'].dt.year==ntb['FKYC_DATE'].dt.year))]

    ntb_1mnth_cnt=ntb_1mnth['ACCNT_NUM'].nunique()
    ntb_1mnth_val=ntb_1mnth['INITIAL_DEPST_AMT'].sum()/10000000
    
    ntb_gt1mnth=ntb[ntb['ACCNT_OPN_DATE']>=(ntb['FKYC_DATE']+ pd.offsets.MonthBegin(1))]
    
    ntb_gt1mnth_cnt=ntb_gt1mnth['ACCNT_NUM'].nunique()
    ntb_gt1mnth_val=ntb_gt1mnth['INITIAL_DEPST_AMT'].sum()/10000000

    
    etb_val=acqui[acqui['FKYC_DATE'].isna()]['INITIAL_DEPST_AMT'].sum()/10000000
    etb_cnt=acqui[acqui['FKYC_DATE'].isna()]['ACCNT_NUM'].nunique()

    scc=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon) &(~df['APPL_USER_2'].isna())]

  
    scc_cnt=scc['ACCNT_NUM'].nunique()
    scc_val=scc['INITIAL_DEPST_AMT'].sum()/10000000


    
    df_value.loc[len(df_value.index)] = [yr,mon,ntb_1mnth_cnt,ntb_1mnth_val,ntb_gt1mnth_cnt,ntb_gt1mnth_val,etb_cnt,etb_val,scc_cnt,scc_val]

    # acqui=acqui[(acqui['fkyc_date'].isna())|(acqui['fkyc_date']<fkyc_limit)]
    

202502
2
2025
(314002, 15)
202503
3
2025
(366391, 15)


In [25]:
df_value

,Year,Month,Acqui_ntb_0_1_cnt,Acqui_ntb_0_1_val,Acqui_ntb_2_12_cnt,Acqui_ntb_2_12_val,Acqui_eTB_cnt,Acqui_eTB_val,SCC cnt,SCC val
0,2025.000,2.000,1258.000,6.522,46105.000,120.954,266631.000,878.000,0.000,0.000
1,2025.000,3.000,2766.000,15.252,36474.000,103.999,327129.000,1104.823,0.000,0.000


In [18]:
df_parent.columns

Index(['CRN', 'ACCNT_NUM', 'ACCNT_OPN_DATE', 'ACCNT_MATURITY_DATE',
       'ACCNT_CLS_DATE', 'RNWAL_DATE', 'INITIAL_DEPST_AMT', 'ACCNT_BAL_IN_BC',
       'PRODCT_CODE', 'MTHID', 'TENOR_IN_DAYS', 'FKYC_DATE', 'APPL_USER_2',
       'DATE_OPENED_G', 'APP_ACCNT_NUM'],
      dtype='object')

In [35]:
df_value=pd.DataFrame(columns=['MTHID','FD','RD','FFD'])

for i in df_parent.MTHID.unique():
    print(i)
    mon=i%100
    yr=int(i/100)
    print(mon)
    print(yr)

    # mthid=yr*100+mon
    df=df_parent[(df_parent['MTHID']==i) ]

    
    
    ##Only for ETB- for others- comment SCC line and fkyc not needed
    df_fd=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]
    df_rd=df[(df['PRODCT_CODE'].str.contains('RD'))]
    df_ffd=df[(df['PRODCT_CODE'].str.contains('FF'))]
    #remove SCC in needed
    # df=df[~df['ACCNT_NUM'].isin(scc.APPL_USER_2.unique())]
    
    # df=df[df['INITIAL_DEPST_AMT']<20000000]

    df_value.loc[len(df_value.index)] = [i,df_fd['ACCNT_BAL_IN_BC'].sum()/10000000,df_rd['ACCNT_BAL_IN_BC'].sum()/10000000,df_ffd['ACCNT_BAL_IN_BC'].sum()/10000000]



202311
11
2023
202303
3
2023
202309
9
2023
202302
2
2023
202305
5
2023
202310
10
2023
202304
4
2023
202308
8
2023
202307
7
2023
202306
6
2023
202312
12
2023
202301
1
2023


In [36]:
df_value

,MTHID,FD,RD,FFD
0,202311.000,2547.212,202.231,747.171
1,202303.000,1813.337,140.179,120.710
2,202309.000,2409.492,187.676,567.321
3,202302.000,1724.796,136.588,114.207
4,202305.000,2064.433,152.982,134.637
5,202310.000,2474.820,194.754,656.444
6,202304.000,1945.469,146.106,125.438
7,202308.000,2348.146,177.684,460.929
8,202307.000,2269.287,168.748,319.112
9,202306.000,2163.973,160.969,182.462


In [15]:
(df_value[['Year','Month','Acqui_etb','Acqui_NTB']]/10000000).T

,0,1,2,3,4,5,6,7,8,9,10,11,12
Year,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Month,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Acqui_etb,58.153,69.389,84.083,56.965,42.769,37.655,44.026,46.838,39.555,40.161,37.127,40.895,45.257
Acqui_NTB,285.765,307.677,358.987,251.904,292.020,286.225,332.950,353.442,334.207,342.645,335.321,375.731,391.002


In [46]:
#NTB /ETB Maturity


df_parent.columns=df_parent.columns.str.upper()

df_value=pd.DataFrame(columns=['Year','Month','Acqui_ntb','Acqui_eTB'])


for i in [202401,202402,202403,  202404,  202405,202406,  202407,
       202408, 202409, 202410,  202411,202412, 202501 ] :
    print(i)
    mon=i%100
    yr=int(i/100)
    print(mon)
    print(yr)

    # mthid=yr*100+mon
    df=df_parent[(df_parent['MTHID']==i) &(~df_parent['APP_ACCNT_NUM'].isna())]
    # df=df_parent[(df_parent['MTHID']==i)&(~df_parent['APP_ACCNT_NUM'].isna())]
    # df_parent.columns=df_parent.columns.str.upper()
    
    #### for FD
    
    
    ##Only for ETB- for others- comment SCC line and fkyc not needed
    df=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]
    # df=df[(df['PRODCT_CODE'].str.contains('RD'))]

    #remove SCC in needed
    # df=df[~df['ACCNT_NUM'].isin(scc.APPL_USER_2.unique())]
    
    df=df[df['INITIAL_DEPST_AMT']<20000000]
    
    df=df.drop_duplicates('ACCNT_NUM')
    
    
    # df_count=pd.DataFrame(columns=['Year','Month','Acqui','AR_planned','AR_unplanned','On_maturity',' Pre Cls_same_mnth','Pre Cls_prev_mnth','Pre Cls_older'])
    
    # df=pd.merge(df,fkyc,left_on='CRN', right_on='crn',how='left')
    # df=df[(df['fkyc_date'].isna())|(df['fkyc_date']<fkyc_limit)]
    
    
    
    
    
    #Acquisition	opn date in that month init amt sum 	later subtract renewal unplanned

    mat=df[((df['ACCNT_CLS_DATE'].dt.year==yr))& (df['ACCNT_MATURITY_DATE']==df['ACCNT_CLS_DATE']) &(df['ACCNT_CLS_DATE'].dt.month==mon)]

    # acqui=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon)]
    ntb=mat[~mat['FKYC_DATE'].isna()]['INITIAL_DEPST_AMT'].sum()/10000000
    etb=mat[mat['FKYC_DATE'].isna()]['INITIAL_DEPST_AMT'].sum()/10000000

    df_value.loc[len(df_value.index)] = [yr,mon,ntb,etb]

    # acqui=acqui[(acqui['fkyc_date'].isna())|(acqui['fkyc_date']<fkyc_limit)]
    

202401
1
2024
202402
2
2024
202403
3
2024
202404
4
2024
202405
5
2024
202406
6
2024
202407
7
2024
202408
8
2024
202409
9
2024
202410
10
2024
202411
11
2024
202412
12
2024
202501
1
2025


In [47]:
df_value

,Year,Month,Acqui_ntb,Acqui_eTB
0,2024.000,1.000,0.000,0.000
1,2024.000,2.000,0.000,0.000
2,2024.000,3.000,0.133,0.000
3,2024.000,4.000,0.068,0.006
4,2024.000,5.000,0.070,0.008
5,2024.000,6.000,0.049,0.051
6,2024.000,7.000,0.125,0.073
7,2024.000,8.000,0.078,0.130
8,2024.000,9.000,0.160,0.183
9,2024.000,10.000,0.645,0.254


In [48]:
#NTB /ETB Maturity

df_parent['FKYC_DATE']=pd.to_datetime(df_parent['FKYC_DATE'])
df_parent.columns=df_parent.columns.str.upper()

df_value=pd.DataFrame(columns=['Year','Month','Acqui_ntb0-1','Acqui_ntb_2-12','Acqui_eTB'])


for i in [202401,202402,202403,  202404,  202405,202406,  202407,
       202408, 202409, 202410,  202411,202412, 202501 ] :
    print(i)
    mon=i%100
    yr=int(i/100)
    print(mon)
    print(yr)

    # mthid=yr*100+mon
    df=df_parent[(df_parent['MTHID']==i) &(~df_parent['APP_ACCNT_NUM'].isna())]
    # df=df_parent[(df_parent['MTHID']==i)&(~df_parent['APP_ACCNT_NUM'].isna())]
    # df_parent.columns=df_parent.columns.str.upper()
    
    #### for FD
    
    
    ##Only for ETB- for others- comment SCC line and fkyc not needed
    df=df[~((df['PRODCT_CODE'].str.contains('FF')) |(df['PRODCT_CODE'].str.contains('RD')) )]
    # df=df[(df['PRODCT_CODE'].str.contains('RD'))]

    #remove SCC in needed
    # df=df[~df['ACCNT_NUM'].isin(scc.APPL_USER_2.unique())]
    
    df=df[df['INITIAL_DEPST_AMT']<20000000]
    
    df=df.drop_duplicates('ACCNT_NUM')
    
    
    # df_count=pd.DataFrame(columns=['Year','Month','Acqui','AR_planned','AR_unplanned','On_maturity',' Pre Cls_same_mnth','Pre Cls_prev_mnth','Pre Cls_older'])
    
    # df=pd.merge(df,fkyc,left_on='CRN', right_on='crn',how='left')
    # df=df[(df['fkyc_date'].isna())|(df['fkyc_date']<fkyc_limit)]
    
    
    
    
    
    #Acquisition	opn date in that month init amt sum 	later subtract renewal unplanned

    cls=df[((df['ACCNT_CLS_DATE'].dt.year==yr))
      & (df['ACCNT_MATURITY_DATE']>df['ACCNT_CLS_DATE'])
         &(df['ACCNT_CLS_DATE'].dt.month==mon)]

    # acqui=df[(df['ACCNT_OPN_DATE'].dt.year==yr)&(df['ACCNT_OPN_DATE'].dt.month==mon)]
    ntb=cls[~cls['FKYC_DATE'].isna()]['INITIAL_DEPST_AMT'].sum()/10000000
    etb=cls[cls['FKYC_DATE'].isna()]['INITIAL_DEPST_AMT'].sum()/10000000

    df_value.loc[len(df_value.index)] = [yr,mon,ntb,etb]

    # acqui=acqui[(acqui['fkyc_date'].isna())|(acqui['fkyc_date']<fkyc_limit)]
    

202401
1
2024
202402
2
2024
202403
3
2024
202404
4
2024
202405
5
2024
202406
6
2024
202407
7
2024
202408
8
2024
202409
9
2024
202410
10
2024
202411
11
2024
202412
12
2024
202501
1
2025


In [49]:
df_value

,Year,Month,Acqui_ntb,Acqui_eTB
0,2024.000,1.000,0.000,0.000
1,2024.000,2.000,0.001,0.000
2,2024.000,3.000,0.321,0.047
3,2024.000,4.000,0.729,0.053
4,2024.000,5.000,0.676,0.532
5,2024.000,6.000,0.849,0.325
6,2024.000,7.000,1.192,0.619
7,2024.000,8.000,1.243,1.072
8,2024.000,9.000,1.224,1.230
9,2024.000,10.000,1.672,2.281
